In [1]:
!pip install transformers

In [0]:
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")
model = AutoModel.from_pretrained("dbmdz/bert-base-turkish-cased")

In [0]:
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm

In [0]:
data = pd.read_csv("dataset.csv")

In [0]:
data = data[:2100]
data['labels'] = pd.factorize(data.category)[0]
del data['category']

In [0]:
tokenized_input = data['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True,max_length=256)))

In [0]:
max_len = 256
for i in tokenized_input.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized_input.values])

In [0]:
attention_masks  = np.where(padded != 0, 1, 0)

In [0]:
input_ids = torch.tensor(padded)  
attention_masks = torch.tensor(attention_masks)

In [0]:
with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_masks)

In [0]:
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
features = last_hidden_states[0][:,0,:].numpy()
labels = data["labels"]
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(labels)
labels = encoder.transform(labels)
labels = np_utils.to_categorical(labels)
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout

In [107]:
#test_labels2 = test_labels.reshape(-1,1)
#train_labels2 = train_labels.reshape(-1,1)
train_features[30][26]


-1.306352

In [115]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout

classifier = Sequential()
classifier.add(Dense(units=256,activation='relu',input_shape=(768,)))
classifier.add(Dense(units=256,activation='relu'))
classifier.add(Dropout(0.2))
classifier.add(Dense(units=256,activation='relu'))
classifier.add(Dense(units=256,activation='relu'))
classifier.add(Dropout(0.2))
classifier.add(Dense(units=256,activation='relu'))
classifier.add(Dense(units=256,activation='relu'))
classifier.add(Dropout(0.2))
classifier.add(Dense(units=256,activation='relu'))
classifier.add(Dense(units=256,activation='relu'))


classifier.add(Dense(3,activation='softmax'))

classifier.compile(optimizer='adam',
              loss='binary_crossentropy',metrics=['accuracy'])

classifier.fit(train_features, train_labels,
          batch_size=32,
          epochs=28,
          validation_data=(test_features, test_labels))


Epoch 1/28
50/50 [==============================] - 1s 10ms/step - loss: 0.6239 - accuracy: 0.3816 - val_loss: 0.4660 - val_accuracy: 0.5905
Epoch 2/28
50/50 [==============================] - 0s 7ms/step - loss: 0.4763 - accuracy: 0.6330 - val_loss: 0.3320 - val_accuracy: 0.8114
Epoch 3/28
50/50 [==============================] - 0s 8ms/step - loss: 0.3730 - accuracy: 0.7460 - val_loss: 0.2987 - val_accuracy: 0.8152
Epoch 4/28
50/50 [==============================] - 0s 7ms/step - loss: 0.3426 - accuracy: 0.7898 - val_loss: 0.5535 - val_accuracy: 0.6114
Epoch 5/28
50/50 [==============================] - 0s 8ms/step - loss: 0.4362 - accuracy: 0.6768 - val_loss: 0.5614 - val_accuracy: 0.7124
Epoch 6/28
50/50 [==============================] - 0s 8ms/step - loss: 0.3242 - accuracy: 0.8197 - val_loss: 0.2499 - val_accuracy: 0.8362
Epoch 7/28
50/50 [==============================] - 0s 7ms/step - loss: 0.2173 - accuracy: 0.8838 - val_loss: 0.1790 - val_accuracy: 0.9105
Epoch 8/28
50/50 [=

In [100]:
vocab_size = tokenizer.vocab_size
print(vocab_size)

32000
